# MRZ CRNN Training (v3: High Accuracy)

MRZ (Machine Readable Zone) 用の軽量 CRNN モデルを学習する。

## v3 改善点
- **Depthwise Separable Conv**: 計算量削減 + 文字幅感度向上
- **縦横比保護**: 拡張時に 0/O の識別特徴を維持
- **横解像度アップ**: 280 → 320 px で文字幅の違いを保持
- **データ量5倍**: 10,000 → 50,000 サンプル
- **Curriculum Learning**: 段階的に拡張強度を上げる
- **混同文字重点学習**: 0/O, 1/I, 8/B などを多く含むサンプル

## 概要
- **アーキテクチャ**: Depthwise Separable CNN + BiLSTM + Self-Attention + CTC
- **入力**: 32x320 グレースケール画像
- **出力**: 44文字の MRZ テキスト
- **目標**: CER < 0.1%, Accuracy >= 99%

## 1. セットアップ

In [ ]:
# GPU 確認
!nvidia-smi

In [ ]:
# 必要なライブラリは Colab に事前インストール済み
import torch
import torch.nn as nn
import numpy as np
from pathlib import Path
import random
import time

# cuDNN最適化（低リスク高速化）
torch.backends.cudnn.benchmark = True  # 最適なアルゴリズム自動選択
torch.backends.cudnn.deterministic = False  # 再現性より速度優先

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"cuDNN benchmark: {torch.backends.cudnn.benchmark}")

## 2. OCR-B フォント準備

MRZ は **OCR-B フォント** で印刷されている（国際標準）。
合成データも必ず OCR-B を使用する。

In [ ]:
# OCR-B フォントをダウンロード（MRZ 標準フォント）
import urllib.request
import os

# jsDelivr CDN経由でダウンロード（安定したソース）
# 元ソース: https://github.com/jaycee723/ocr-b
OCRB_FONT_URL = "https://cdn.jsdelivr.net/gh/jaycee723/ocr-b/dist/OCR-B.otf"
OCRB_FONT_PATH = "/tmp/OCRB.otf"

if not os.path.exists(OCRB_FONT_PATH):
    print("OCR-B フォントをダウンロード中...")
    urllib.request.urlretrieve(OCRB_FONT_URL, OCRB_FONT_PATH)
    print(f"✅ ダウンロード完了: {OCRB_FONT_PATH}")
else:
    print(f"✅ OCR-B フォント準備済み: {OCRB_FONT_PATH}")

# フォント読み込みテスト
from PIL import ImageFont
try:
    test_font = ImageFont.truetype(OCRB_FONT_PATH, 24)
    print(f"✅ フォント読み込み成功")
except Exception as e:
    raise RuntimeError(f"OCR-B フォントの読み込みに失敗: {e}")

In [ ]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageEnhance
import string
import io

# MRZ で使用する文字セット
MRZ_CHARS = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789<"

# 混同しやすい文字ペア（v3: 重点学習用）
CONFUSABLE_CHARS = "0ODQ1IL8B5S"


def generate_random_mrz_line(hard_mode: bool = False) -> str:
    """
    ランダムな MRZ 行（44文字）を生成
    
    Args:
        hard_mode: True の場合、混同しやすい文字を多く含む
    """
    if hard_mode:
        def pick_char(choices):
            if random.random() < 0.3 and any(c in CONFUSABLE_CHARS for c in choices):
                confusable = [c for c in choices if c in CONFUSABLE_CHARS]
                if confusable:
                    return random.choice(confusable)
            return random.choice(choices)
        pick_chars = lambda choices, k: "".join(pick_char(choices) for _ in range(k))
    else:
        pick_chars = lambda choices, k: "".join(random.choices(choices, k=k))
    
    if random.random() < 0.5:
        doc_type = random.choice(["P", "I", "A", "C"])
        country = pick_chars(string.ascii_uppercase, 3)
        name_len = random.randint(20, 35)
        name = pick_chars(string.ascii_uppercase + "<", name_len)
        line = f"{doc_type}<{country}{name}"
        line = line[:44].ljust(44, "<")
    else:
        passport_no = pick_chars(string.ascii_uppercase + string.digits, 9)
        check1 = random.choice(string.digits)
        nationality = pick_chars(string.ascii_uppercase, 3)
        dob = pick_chars(string.digits, 6)
        check2 = random.choice(string.digits)
        sex = random.choice(["M", "F", "<"])
        expiry = pick_chars(string.digits, 6)
        check3 = random.choice(string.digits)
        optional = pick_chars(string.ascii_uppercase + string.digits + "<", 14)
        check4 = random.choice(string.digits)
        line = f"{passport_no}{check1}{nationality}{dob}{check2}{sex}{expiry}{check3}{optional}{check4}"
        line = line[:44]
    
    return line


def apply_augmentation(img: Image.Image, intensity: float = 1.0) -> Image.Image:
    """
    データ拡張（v3: 縦横比保護 + Curriculum Learning対応）
    
    重要: OCR-B フォントでは 0 と O の違いが縦横比で表現される。
    拡張時に縦横比を歪めると、この識別特徴が失われる。
    → アフィン変換で等倍スケーリングを使用
    """
    # ブラー
    if random.random() < 0.3 * intensity:
        img = img.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.3, 1.0)))
    
    # 回転（縦横比を保持）
    if random.random() < 0.5 * intensity:
        angle = random.uniform(-2, 2) * intensity
        img = img.rotate(angle, fillcolor=255, resample=Image.BILINEAR)
    
    # アフィン変換（v3: 縦横比を保護）
    if random.random() < 0.2 * intensity:
        w, h = img.size
        
        # 等倍スケーリング: x と y を同じ倍率で変換
        # これにより 0(狭い) と O(広い) の違いが保持される
        scale = 1 + random.uniform(-0.02, 0.02) * intensity
        
        # せん断（shear）は横方向のみ許容（文字幅に影響しにくい）
        shear_x = random.uniform(-0.01, 0.01) * intensity
        
        # 平行移動
        tx = random.uniform(-0.02, 0.02) * w * intensity
        ty = random.uniform(-0.02, 0.02) * h * intensity
        
        coeffs = [
            scale,      # x スケール
            shear_x,    # x せん断
            tx,         # x 平行移動
            0,          # y せん断 (0 で縦横比保護)
            scale,      # y スケール (x と同じ)
            ty,         # y 平行移動
        ]
        img = img.transform((w, h), Image.AFFINE, coeffs, fillcolor=255)
    
    # 明るさ
    if random.random() < 0.4 * intensity:
        factor = 1.0 + (random.uniform(-0.2, 0.2) * intensity)
        img = ImageEnhance.Brightness(img).enhance(factor)
    
    # コントラスト
    if random.random() < 0.4 * intensity:
        factor = 1.0 + (random.uniform(-0.2, 0.2) * intensity)
        img = ImageEnhance.Contrast(img).enhance(factor)
    
    # JPEG圧縮アーティファクト
    if random.random() < 0.2 * intensity:
        quality = int(95 - 25 * intensity)
        buffer = io.BytesIO()
        img.save(buffer, format='JPEG', quality=quality)
        buffer.seek(0)
        img = Image.open(buffer).convert('L')
    
    return img


def render_mrz_image(
    text: str,
    height: int = 32,
    font_size: int = 24,
    augment_intensity: float = 1.0
) -> np.ndarray:
    """MRZ テキストを OCR-B フォントで画像にレンダリング"""
    # OCR-B フォントのみ使用（MRZ標準、フォールバックなし）
    font = ImageFont.truetype(OCRB_FONT_PATH, font_size)
    
    dummy_img = Image.new("L", (1, 1))
    dummy_draw = ImageDraw.Draw(dummy_img)
    bbox = dummy_draw.textbbox((0, 0), text, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    
    padding = 4
    img_width = text_width + padding * 2
    img = Image.new("L", (img_width, height), color=255)
    draw = ImageDraw.Draw(img)
    
    y_offset = (height - text_height) // 2
    draw.text((padding, y_offset), text, font=font, fill=0)
    
    if augment_intensity > 0:
        img = apply_augmentation(img, intensity=augment_intensity)
    
    img_array = np.array(img)
    
    if augment_intensity > 0 and random.random() < 0.6 * augment_intensity:
        sigma = random.uniform(3, 10) * augment_intensity
        noise = np.random.normal(0, sigma, img_array.shape)
        img_array = np.clip(img_array + noise, 0, 255).astype(np.uint8)
    
    return img_array


# テスト
print("v3: OCR-B + 縦横比保護 + Curriculum Learning + Hard Mode")
print("\n通常サンプル:")
print(f"  {generate_random_mrz_line(hard_mode=False)}")
print("\n混同文字重点サンプル:")
hard = generate_random_mrz_line(hard_mode=True)
print(f"  {hard}")
print(f"  混同文字数: {sum(1 for c in hard if c in CONFUSABLE_CHARS)}")

In [ ]:
# サンプル画像を表示
import matplotlib.pyplot as plt

fig, axes = plt.subplots(3, 1, figsize=(12, 4))
for i, ax in enumerate(axes):
    text = generate_random_mrz_line()
    img = render_mrz_image(text)
    ax.imshow(img, cmap="gray")
    ax.set_title(text, fontsize=8)
    ax.axis("off")
plt.tight_layout()
plt.show()

## 3. Dataset & DataLoader

In [ ]:
from torch.utils.data import Dataset, DataLoader

# 文字セット定義
CHARS = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789<"
CHAR_TO_IDX = {c: i for i, c in enumerate(CHARS)}
IDX_TO_CHAR = {i: c for i, c in enumerate(CHARS)}
NUM_CLASSES = len(CHARS) + 1  # +1 for CTC blank

print(f"文字数: {len(CHARS)}")
print(f"クラス数 (blank含む): {NUM_CLASSES}")


def encode_text(text: str) -> list:
    """テキストを数値インデックスに変換"""
    return [CHAR_TO_IDX[c] for c in text if c in CHAR_TO_IDX]


def decode_output(indices: list) -> str:
    """
    CTC 出力をテキストにデコード
    連続する同一インデックスと blank を除去
    """
    result = []
    prev_idx = -1
    for idx in indices:
        if idx == len(CHARS):  # blank
            prev_idx = idx
            continue
        if idx != prev_idx and idx < len(CHARS):
            result.append(IDX_TO_CHAR[idx])
        prev_idx = idx
    return "".join(result)


class SyntheticMRZDataset(Dataset):
    """
    合成 MRZ データセット (v3: Curriculum Learning + Hard Mode対応)
    
    オンラインでランダムに MRZ 画像を生成する。
    - augment_intensity: 拡張強度（Curriculum Learningで段階的に上げる）
    - hard_ratio: 混同文字重点サンプルの割合
    """
    
    def __init__(
        self, 
        num_samples: int, 
        max_width: int = 280,
        augment_intensity: float = 1.0,
        hard_ratio: float = 0.3
    ):
        self.num_samples = num_samples
        self.max_width = max_width
        self.augment_intensity = augment_intensity
        self.hard_ratio = hard_ratio
    
    def set_augment_intensity(self, intensity: float):
        """Curriculum Learning 用: 拡張強度を動的に変更"""
        self.augment_intensity = intensity
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        # hard_ratio の確率で混同文字重点サンプルを生成
        hard_mode = random.random() < self.hard_ratio
        text = generate_random_mrz_line(hard_mode=hard_mode)
        image = render_mrz_image(text, augment_intensity=self.augment_intensity)
        
        # 正規化 (0-1)
        image = image.astype(np.float32) / 255.0
        
        # パディング（幅を max_width に統一）
        h, w = image.shape
        if w < self.max_width:
            pad_w = self.max_width - w
            image = np.pad(image, ((0, 0), (0, pad_w)), constant_values=1.0)
        elif w > self.max_width:
            image = image[:, :self.max_width]
        
        # テンソルに変換 (1, H, W)
        image_tensor = torch.from_numpy(image).unsqueeze(0)
        label = encode_text(text)
        
        return {
            "image": image_tensor,
            "label": torch.tensor(label, dtype=torch.long),
            "label_length": len(label),
            "text": text
        }


def collate_fn(batch):
    """バッチをまとめる（CTC Loss 用）"""
    images = torch.stack([item["image"] for item in batch])
    labels = torch.cat([item["label"] for item in batch])
    label_lengths = torch.tensor([item["label_length"] for item in batch])
    texts = [item["text"] for item in batch]
    return {
        "images": images,
        "labels": labels,
        "label_lengths": label_lengths,
        "texts": texts
    }


print("Dataset: v3 (Curriculum Learning + Hard Mode対応)")

## 4. CRNN モデル

In [ ]:
class DepthwiseSeparableConv(nn.Module):
    """
    Depthwise Separable Convolution (MobileNet スタイル)
    
    通常の Conv2d を2つのステップに分解:
    1. Depthwise Conv: 各チャネルを独立に畳み込み（空間特徴抽出）
    2. Pointwise Conv: 1x1 でチャネル間を結合
    
    効果:
    - 計算量: 約 1/8 に削減
    - 横方向の特徴を保持しやすい（文字幅の識別に有利）
    """
    
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int = 3, 
                 stride: int = 1, padding: int = 1):
        super().__init__()
        
        # Depthwise Conv: 各チャネルを独立に処理
        # groups=in_channels で各チャネルに別々のフィルタを適用
        self.depthwise = nn.Conv2d(
            in_channels, in_channels, 
            kernel_size=kernel_size, 
            stride=stride, 
            padding=padding, 
            groups=in_channels,  # ← 核心: チャネルごとに独立
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(in_channels)
        
        # Pointwise Conv: 1x1 でチャネル間を結合
        self.pointwise = nn.Conv2d(
            in_channels, out_channels, 
            kernel_size=1, 
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        # Depthwise: 空間方向の特徴抽出（横幅の違いを保持）
        x = self.depthwise(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        # Pointwise: チャネル間の結合
        x = self.pointwise(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        return x


class EfficientCRNN(nn.Module):
    """
    Efficient CRNN with Depthwise Separable Convolution (v3)
    
    アーキテクチャ:
    - Depthwise Separable CNN: 軽量かつ横方向特徴を保持
    - BiLSTM: シーケンスモデリング
    - Self-Attention: 文字間依存関係
    
    v3 改善点:
    - Depthwise Separable Conv で計算量削減 + 文字幅感度向上
    - 最初の層は通常 Conv（低レベル特徴は混ぜる方が良い）
    - Squeeze-and-Excitation 的なチャネル注意を追加可能
    
    入力: (B, 1, 32, W) - グレースケール画像
    出力: (T, B, 38) - 各タイムステップの文字確率
    """
    
    def __init__(self, num_classes: int = 38, hidden_size: int = 128, dropout: float = 0.2):
        super().__init__()
        
        self.hidden_size = hidden_size
        
        # CNN Backbone (Depthwise Separable)
        self.cnn = nn.Sequential(
            # Block 1: 最初の層は通常 Conv（低レベル特徴抽出）
            # 32 -> 16
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # Block 2: Depthwise Separable (16 -> 8)
            DepthwiseSeparableConv(32, 64),
            nn.MaxPool2d(2, 2),
            
            # Block 3: Depthwise Separable (8 -> 4)
            DepthwiseSeparableConv(64, 128),
            nn.MaxPool2d((2, 1)),  # 高さのみ縮小、幅は保持
            
            # Block 4: Depthwise Separable (4 -> 2)
            DepthwiseSeparableConv(128, 256),
            nn.MaxPool2d((2, 1)),  # 高さのみ縮小
            
            # Block 5: 最終層 (2 -> 1)
            nn.Conv2d(256, 256, kernel_size=(2, 1)),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )
        
        # BiLSTM
        self.lstm = nn.LSTM(
            input_size=256,
            hidden_size=hidden_size,
            num_layers=2,
            bidirectional=True,
            batch_first=False,
            dropout=dropout
        )
        
        # Self-Attention
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_size * 2,
            num_heads=4,
            dropout=dropout,
            batch_first=False
        )
        
        # LayerNorm + Dropout
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        self.dropout = nn.Dropout(dropout)
        
        # 出力層
        self.fc = nn.Linear(hidden_size * 2, num_classes)
    
    def forward(self, x):
        # CNN 特徴抽出: (B, 1, 32, W) -> (B, 256, 1, W')
        features = self.cnn(x)
        
        # 形状変換: (B, C, 1, W') -> (W', B, C) = (T, B, C)
        b, c, h, w = features.shape
        features = features.squeeze(2)
        features = features.permute(2, 0, 1)  # (T, B, C)
        
        # BiLSTM
        lstm_out, _ = self.lstm(features)
        
        # Self-Attention + Residual
        attn_out, _ = self.attention(lstm_out, lstm_out, lstm_out)
        out = self.layer_norm(lstm_out + attn_out)
        out = self.dropout(out)
        
        # 出力
        output = self.fc(out)
        output = torch.log_softmax(output, dim=2)
        
        return output


# 旧モデル（比較用に残す）
class AttentionCRNN(nn.Module):
    """v2: 通常の Conv2d + Attention"""
    
    def __init__(self, num_classes: int = 38, hidden_size: int = 128, dropout: float = 0.2):
        super().__init__()
        self.hidden_size = hidden_size
        
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.Conv2d(256, 256, kernel_size=(2, 1)), nn.BatchNorm2d(256), nn.ReLU(),
        )
        
        self.lstm = nn.LSTM(256, hidden_size, 2, bidirectional=True, batch_first=False, dropout=dropout)
        self.attention = nn.MultiheadAttention(hidden_size * 2, 4, dropout=dropout, batch_first=False)
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
    
    def forward(self, x):
        features = self.cnn(x)
        b, c, h, w = features.shape
        features = features.squeeze(2).permute(2, 0, 1)
        lstm_out, _ = self.lstm(features)
        attn_out, _ = self.attention(lstm_out, lstm_out, lstm_out)
        out = self.layer_norm(lstm_out + attn_out)
        out = self.dropout(out)
        output = self.fc(out)
        return torch.log_softmax(output, dim=2)


def get_model_info(model):
    """モデル情報を取得"""
    total_params = sum(p.numel() for p in model.parameters())
    param_size = sum(p.numel() * p.element_size() for p in model.parameters())
    buffer_size = sum(b.numel() * b.element_size() for b in model.buffers())
    size_mb = (param_size + buffer_size) / 1024 / 1024
    return {"total_params": total_params, "size_mb": size_mb}


# モデル比較
print("=" * 50)
print("モデル比較")
print("=" * 50)

# v2: 通常 Conv
model_v2 = AttentionCRNN(num_classes=NUM_CLASSES, dropout=0.2)
info_v2 = get_model_info(model_v2)

# v3: Depthwise Separable Conv
model_v3 = EfficientCRNN(num_classes=NUM_CLASSES, dropout=0.2)
info_v3 = get_model_info(model_v3)

print(f"AttentionCRNN (v2):")
print(f"  パラメータ数: {info_v2['total_params']:,}")
print(f"  モデルサイズ: {info_v2['size_mb']:.2f} MB")

print(f"\nEfficientCRNN (v3 Depthwise Separable):")
print(f"  パラメータ数: {info_v3['total_params']:,}")
print(f"  モデルサイズ: {info_v3['size_mb']:.2f} MB")

reduction = (1 - info_v3['total_params'] / info_v2['total_params']) * 100
print(f"\n→ パラメータ削減: {reduction:.1f}%")

# 推論テスト
x = torch.randn(1, 1, 32, 280)
output_v2 = model_v2(x)
output_v3 = model_v3(x)
print(f"\n入力: {x.shape}")
print(f"出力 (v2): {output_v2.shape}")
print(f"出力 (v3): {output_v3.shape}")

# v3 を使用
model = model_v3
print(f"\n✅ EfficientCRNN (v3) を使用")

## 5. 学習

In [ ]:
import os
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR

# ハイパーパラメータ (v3: High Accuracy)
BATCH_SIZE = 64   # 安定性優先
EPOCHS = 150      # 100 → 150 (収束までの余裕)
LR = 1e-3
WEIGHT_DECAY = 0.01
TRAIN_SAMPLES = 50000   # 10,000 → 50,000 (5倍)
VAL_SAMPLES = 2000      # 検証サンプルも増加
MAX_WIDTH = 320         # 280 → 320 (横解像度アップ: 文字幅の違いを保持)

# Curriculum Learning パラメータ
CURRICULUM_STAGES = [
    # (開始epoch, augment_intensity, hard_ratio)
    (1, 0.3, 0.1),    # Stage 1: 簡単なサンプルで基礎学習
    (30, 0.6, 0.2),   # Stage 2: 中程度の拡張
    (60, 1.0, 0.3),   # Stage 3: フル拡張 + 混同文字重点
]

# デバイス
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# num_workers: Colab/Kaggle では 0 が最も安定（オンライン生成のため）
NUM_WORKERS = 0
print(f"num_workers: {NUM_WORKERS} (オンライン生成のため)")

# データセット（初期は低強度）
initial_intensity, initial_hard_ratio = CURRICULUM_STAGES[0][1], CURRICULUM_STAGES[0][2]
train_dataset = SyntheticMRZDataset(
    TRAIN_SAMPLES, MAX_WIDTH,
    augment_intensity=initial_intensity,
    hard_ratio=initial_hard_ratio
)
val_dataset = SyntheticMRZDataset(VAL_SAMPLES, MAX_WIDTH, augment_intensity=1.0, hard_ratio=0.3)

# DataLoader
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=NUM_WORKERS,
    pin_memory=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

print(f"Train samples: {TRAIN_SAMPLES:,} (5x increase)")
print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"MAX_WIDTH: {MAX_WIDTH} (横解像度アップ)")

# モデル (v3: EfficientCRNN with Depthwise Separable Conv)
model = EfficientCRNN(num_classes=NUM_CLASSES, dropout=0.2).to(device)
info = get_model_info(model)
print(f"\nモデル: EfficientCRNN (Depthwise Separable)")
print(f"パラメータ数: {info['total_params']:,}")
print(f"モデルサイズ: {info['size_mb']:.2f} MB")
print("ℹ️ torch.compile() 無効 (Colab/Kaggle 環境)")

criterion = nn.CTCLoss(blank=NUM_CLASSES - 1, zero_infinity=True)
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

# OneCycleLR: max_lr を控えめに設定（CTC collapse 防止）
scheduler = OneCycleLR(
    optimizer,
    max_lr=LR * 3,  # 0.003 (CTC collapse 防止)
    epochs=EPOCHS,
    steps_per_epoch=len(train_loader),
    pct_start=0.1,
    anneal_strategy='cos'
)

print(f"\n[v3 High Accuracy 設定]")
print(f"バッチサイズ: {BATCH_SIZE}")
print(f"エポック数: {EPOCHS}")
print(f"学習サンプル: {TRAIN_SAMPLES:,}")
print(f"max_lr: {LR * 3:.4f}")
print(f"Curriculum Learning: 有効")
print(f"  Stage 1 (epoch 1-29):  intensity=0.3, hard_ratio=0.1")
print(f"  Stage 2 (epoch 30-59): intensity=0.6, hard_ratio=0.2")
print(f"  Stage 3 (epoch 60+):   intensity=1.0, hard_ratio=0.3")

In [ ]:
# PyTorch 2.x 対応の AMP API
from torch.amp import autocast, GradScaler

def train_epoch(model, dataloader, criterion, optimizer, scheduler, device, scaler):
    """
    1エポック分の学習（AMP対応で高速化）
    
    Mixed Precision Training:
    - forward passをFP16で実行（高速化 + メモリ削減）
    - backward passはFP32で実行（精度維持）
    - GradScalerでgradientのunderflow防止
    """
    model.train()
    total_loss = 0.0
    
    for batch in dataloader:
        images = batch["images"].to(device)
        labels = batch["labels"].to(device)
        label_lengths = batch["label_lengths"]
        
        optimizer.zero_grad()
        
        # Mixed Precision: forward pass をFP16で実行
        with autocast('cuda'):
            outputs = model(images)  # (T, B, C)
            T, B, C = outputs.shape
            input_lengths = torch.full((B,), T, dtype=torch.long)
            loss = criterion(outputs, labels, input_lengths, label_lengths)
        
        # Backward with gradient scaling
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)


def validate(model, dataloader, device):
    """検証"""
    model.eval()
    total_chars = 0
    total_errors = 0
    correct = 0
    total = 0
    samples = []
    
    with torch.no_grad():
        for batch in dataloader:
            images = batch["images"].to(device)
            texts = batch["texts"]
            
            with autocast('cuda'):
                outputs = model(images)
            
            for i, text in enumerate(texts):
                probs = outputs[:, i, :]
                pred_indices = probs.argmax(dim=1).cpu().tolist()
                pred_text = decode_output(pred_indices)
                
                errors = sum(1 for a, b in zip(text, pred_text) if a != b)
                errors += abs(len(text) - len(pred_text))
                total_chars += len(text)
                total_errors += errors
                
                if text == pred_text:
                    correct += 1
                total += 1
                
                if len(samples) < 5:
                    samples.append({"gt": text, "pred": pred_text, "match": text == pred_text})
    
    cer = (total_errors / total_chars) * 100 if total_chars > 0 else 0
    accuracy = (correct / total) * 100 if total > 0 else 0
    
    return {"cer": cer, "accuracy": accuracy, "samples": samples}

In [ ]:
# 学習ループ (v3: Curriculum Learning + AMP)
print("=" * 60)
print("学習開始 (v3: Curriculum Learning + Mixed Precision)")
print("=" * 60)

# GradScaler for AMP (PyTorch 2.x API)
scaler = GradScaler('cuda')

best_cer = float("inf")
best_epoch = 0
VAL_FREQUENCY = 5  # 5エポックごとに検証

history = {"train_loss": [], "val_cer": [], "val_acc": [], "lr": []}
current_stage = 0  # Curriculum Learning のステージ追跡

start_time = time.time()

for epoch in range(1, EPOCHS + 1):
    epoch_start = time.time()
    
    # === Curriculum Learning: ステージ更新 ===
    # 現在のエポックに応じて拡張強度と混同文字比率を調整
    for stage_idx, (stage_epoch, intensity, hard_ratio) in enumerate(CURRICULUM_STAGES):
        if epoch >= stage_epoch:
            if stage_idx > current_stage:
                current_stage = stage_idx
                train_dataset.augment_intensity = intensity
                train_dataset.hard_ratio = hard_ratio
                print(f"\n📈 Curriculum Stage {stage_idx + 1}: intensity={intensity}, hard_ratio={hard_ratio}")
    
    current_lr = optimizer.param_groups[0]['lr']
    history["lr"].append(current_lr)
    
    # 学習
    train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler, device, scaler)
    history["train_loss"].append(train_loss)
    
    epoch_time = time.time() - epoch_start
    
    # 検証（5エポックごと or 最初の10エポック or 最終エポック or ステージ変更直後）
    is_stage_change = any(epoch == s[0] for s in CURRICULUM_STAGES)
    if epoch <= 10 or epoch % VAL_FREQUENCY == 0 or epoch == EPOCHS or is_stage_change:
        val_result = validate(model, val_loader, device)
        history["val_cer"].append(val_result["cer"])
        history["val_acc"].append(val_result["accuracy"])
        
        stage_info = f"[S{current_stage + 1}]" if current_stage > 0 else ""
        print(f"Epoch {epoch:3d}/{EPOCHS} {stage_info} | "
              f"Loss: {train_loss:.4f} | "
              f"CER: {val_result['cer']:.2f}% | "
              f"Acc: {val_result['accuracy']:.1f}% | "
              f"LR: {current_lr:.2e} | "
              f"Time: {epoch_time:.1f}s")
        
        # ベストモデル保存
        if val_result["cer"] < best_cer:
            best_cer = val_result["cer"]
            best_epoch = epoch
            torch.save({
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "cer": best_cer,
                "accuracy": val_result["accuracy"]
            }, "best_model.pth")
            print(f"  -> Best model saved (CER: {best_cer:.2f}%)")
        
        # サンプル表示（30エポックごと）
        if epoch % 30 == 0:
            print("\n  サンプル予測:")
            for s in val_result["samples"][:3]:
                mark = "✓" if s["match"] else "✗"
                print(f"    GT:   {s['gt']}")
                print(f"    Pred: {s['pred']} [{mark}]")
            print()

total_time = time.time() - start_time
print("\n" + "=" * 60)
print("学習完了 (v3)")
print("=" * 60)
print(f"総学習時間: {total_time:.1f}秒 ({total_time/60:.1f}分)")
print(f"ベスト CER: {best_cer:.2f}% (Epoch {best_epoch})")
print(f"目標: CER < 0.1%, Accuracy >= 99%")

## 6. ONNX エクスポート

In [ ]:
# ベストモデルをロード
checkpoint = torch.load("best_model.pth")
model.load_state_dict(checkpoint["model_state_dict"])
model.training = False  # set to inference mode

print(f"Loaded model from epoch {checkpoint['epoch']}")
print(f"CER: {checkpoint['cer']:.2f}%, Accuracy: {checkpoint['accuracy']:.1f}%")

In [ ]:
# ONNX エクスポート & 保存（エラーが発生しても出力を確保）
import shutil
import os

# ONNX エクスポート
model_cpu = model.cpu()
model_cpu.eval()
dummy_input = torch.randn(1, 1, 32, MAX_WIDTH)  # MAX_WIDTH を使用

torch.onnx.export(
    model_cpu,
    dummy_input,
    "mrz_crnn.onnx",
    input_names=["image"],
    output_names=["output"],
    dynamic_axes={
        "image": {0: "batch", 3: "width"},
        "output": {0: "seq_len", 1: "batch"}
    },
    opset_version=17,
    dynamo=False
)

onnx_size = os.path.getsize("mrz_crnn.onnx") / 1024 / 1024
print(f"✅ ONNX エクスポート完了: {onnx_size:.2f} MB")

# 即座に出力ディレクトリに保存（Kaggle/Colab 両対応）
IS_KAGGLE = os.environ.get('KAGGLE_KERNEL_RUN_TYPE') is not None

if IS_KAGGLE:
    output_dir = "/kaggle/working"
    shutil.copy("best_model.pth", f"{output_dir}/best_model.pth")
    shutil.copy("mrz_crnn.onnx", f"{output_dir}/mrz_crnn.onnx")
    print(f"✅ Kaggle Output に保存完了")
else:
    print(f"✅ カレントディレクトリに保存済み")

## 7. モデルを保存

Colab: Google Driveに保存  
Kaggle: Output として保存（ダウンロード可能）

## 7. 完了

モデルは自動的に保存されました:
- `best_model.pth`: PyTorch チェックポイント
- `mrz_crnn.onnx`: ONNX モデル（ブラウザ推論用）

Kaggle: Output タブからダウンロード可能